## Algorithm of computing correlation

```
Iterations = 40
Trials = 10
Conditions = 9 (stim x med)
Channels = 27
At the end you should have 40 samples for all statistical comparisons, RSA matrices (Slide 10), performance Comparisons(Slide 11 and 12)
For -> Iterations
	For -> Conditions
		For -> Trials and subjects in Trials4Test
			Average = RSA between Channels within PACs on the inputs
		End ->Trials
		Average = Trials
	End -> Conditions
	Store = Conditions (9 values/matrices)
End -> Iterations
Stat tests on 9 x 40 values/matrices
```

In [39]:
from itertools import combinations_with_replacement

import numpy as np
import pandas as pd
from scipy.signal import resample
from scipy.stats import pearsonr

In [2]:
path_pacs_dataset = "/path-to-pacs-dir/"
df = pd.read_parquet(path_pacs_dataset + "data_merged.parquet")
df.drop_duplicates(subset=['task', 'stimulus', 'medication', 'channel', 'subject', 'trial'], 
                    keep='first', inplace=True, ignore_index=True)
df = df.dropna(subset=["peak_time", "reaction_time"])
print(df.info())
print(df)

In [45]:
path_pacs_dataset = "/path-to-pacs-dir/"
data_tt = pd.read_excel(path_pacs_dataset + "Trials4Test_All_PD.xlsx")
iterations = np.arange(0, len(data_tt))
df['label'] = np.arange(0, len(df))

In [ ]:
def get_ids(itr):
    row = data_tt.iloc[int(iterations[itr])]
    row = pd.DataFrame(row).iloc[1:,:].reset_index()
    row.rename(columns={"index": "subject", itr: "trial"}, inplace=True)
    row["subject"] = row["subject"].apply(lambda rec: float(rec[2]))
    return row

def get_df_itr(itr):
    row = get_ids(itr)
    df_itr = pd.DataFrame([])
    for j, item in row.iterrows():
        temp = df[(df["subject"] == item["subject"]) & (df["trial"] == item["trial"])]
        df_itr = pd.concat([df_itr, temp])
    df_itr = df_itr.drop_duplicates(subset='label', keep="last")
    return df_itr

In [4]:
df['medication'].replace('pdoffmed_gvs8', 'pdoffmed', inplace=True)
df['medication'].replace('hcoffmed_gvs8', 'hcoffmed', inplace=True)
df['medication'].replace('pdonmed_gvs8', 'pdonmed', inplace=True)

In [5]:
df['medication'].replace('hcoffmed', "HC", inplace=True)
df['medication'].replace('pdonmed', "PD1", inplace=True)
df['medication'].replace('pdoffmed', "PD2", inplace=True)

In [6]:
df["stimulus"].replace("sham", "Sham", inplace=True)
df["stimulus"].replace("stim7", "GVS7", inplace=True)
df["stimulus"].replace("stim8", "GVS8", inplace=True)

In [17]:
## Find the region of interests

df["beta_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, 16:].ravel())
df["alpha_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, 8:16].ravel())
df["theta_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, 4:8].ravel())
df["delta_gamma"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[32:45, :4].ravel())

df["alpha_beta"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[16:32, 8:16].ravel())
df["theta_beta"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[16:32, 4:8].ravel())
df["delta_beta"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[16:32, :4].ravel())

df["theta_alpha"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[8:16, 4:8].ravel())
df["delta_alpha"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[8:16, :4].ravel())

df["delta_theta"] = df["pac_values"].apply(lambda rec: rec.reshape(60, 30)[4:8, :4].ravel())

In [18]:
max_length = 182

df["beta_gamma"] = df["beta_gamma"].apply(lambda rec: resample(rec, max_length))
df["alpha_gamma"] = df["alpha_gamma"].apply(lambda rec: resample(rec, max_length))
df["theta_gamma"] = df["theta_gamma"].apply(lambda rec: resample(rec, max_length))
df["delta_gamma"] = df["delta_gamma"].apply(lambda rec: resample(rec, max_length))

df["alpha_beta"] = df["alpha_beta"].apply(lambda rec: resample(rec, max_length))
df["theta_beta"] = df["theta_beta"].apply(lambda rec: resample(rec, max_length))
df["delta_beta"] = df["delta_beta"].apply(lambda rec: resample(rec, max_length))

df["theta_alpha"] = df["theta_alpha"].apply(lambda rec: resample(rec, max_length))
df["delta_alpha"] = df["delta_alpha"].apply(lambda rec: resample(rec, max_length))

df["delta_theta"] = df["delta_theta"].apply(lambda rec: resample(rec, max_length))

In [41]:
rois = ['beta_gamma', 'alpha_gamma', 'theta_gamma', 'delta_gamma',
        'alpha_beta', 'theta_beta', 'delta_beta',
        'theta_alpha', 'delta_alpha', 
        'delta_theta']

In [ ]:
data_corr = []
run_id = 0

for itr in iterations:
    print(itr)
    df_itr = get_df_itr(itr)
    for med in df_itr.medication.unique():
        df_sel = df_itr[df_itr["medication"] == med]
        for sbj in df_sel.subject.unique():
            df_sbj = df_sel[df_sel["subject"] == sbj]
            for stim in df_sbj.stimulus.unique():
                df_stim = df_sbj[df_sbj["stimulus"] == stim]
                for channel in df_stim.channel.unique():
                    df_chnl = df_stim[df_stim["channel"] == channel]
                    for t1, t2 in combinations_with_replacement(df_chnl.trial.unique(), 2):
                        for reg1, reg2 in combinations_with_replacement(rois, 2):
                            x = df_chnl[(df_chnl["trial"] == t1)][reg1].to_numpy()[0]
                            y = df_chnl[(df_chnl["trial"] == t2)][reg2].to_numpy()[0]
                            corr_ = pearsonr(x, y)[0]
                            stats = {
                                    "iteration": itr + 1,
                                    "condition": med,
                                    "stimulus": stim,
                                    "subject": sbj,
                                    "channel": channel,
                                    "trial_1": t1,
                                    "trial_2": t2,
                                    "roi_1": reg1,
                                    "roi_2": reg2,
                                    "pac_1": x,
                                    "pac_2": y,
                                    "correlation": corr_
                                    }
                            data_corr.append(stats)                        
                            run_id = run_id + 1
                            # print('Run id: ', run_id) ---> Wow!!!! increases running time!
                            # clear_output(wait=True) ---> Wow!!!! increases running time!
                print('Run id: ', run_id)

In [ ]:
path_results = "/path-to-results/"
df_corr = pd.DataFrame.from_records(data_corr)
data_corr.to_csv(path_results + "roi_4test_corrs_allChannels.csv", index=False)